# Benchmark load_by_run_spec

The time required to load a dataset using `load_by_run_spec`when compared to the time taken by `load_by_ guid` or `load_by_id` is 100 times higher. This creates a issue if the database is large. Here I have used database or around 10gb.

In [1]:
from qcodes import initialise_or_create_database_at, load_by_id, load_by_guid, load_by_run_spec

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\a-halakh\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\a-halakh\.qcodes\logs\200114-5936-qcodes.log


In [2]:
initialise_or_create_database_at(r"C:\Users\a-halakh\qt5-experiments-2019-08-29.db")

In [3]:
%timeit load_by_id(1234)

288 ms ± 39.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit load_by_guid('aaaaaaaa-0000-0000-0000-01673807fd53')

365 ms ± 153 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%time load_by_run_spec(captured_run_id=1234)

Wall time: 32.4 s


results #1234@C:\Users\a-halakh\qt5-experiments-2019-08-29.db
-------------------------------------------------------------
scanner_b - array
scanner_pl - array
s_x - array
s_y - array

# Observation: Initial Connection taking longer time

In [6]:
from qcodes.dataset.data_set import get_DB_location,connect

In [7]:
conn = connect(get_DB_location())

In [8]:
%time load_by_run_spec(captured_run_id=1234, conn = conn)

Wall time: 30.2 s


results #1234@C:\Users\a-halakh\qt5-experiments-2019-08-29.db
-------------------------------------------------------------
scanner_b - array
scanner_pl - array
s_x - array
s_y - array

In [9]:
%timeit load_by_id(run_id = 1234, conn = conn)

1.01 ms ± 54 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [10]:
%timeit load_by_guid(guid = 'aaaaaaaa-0000-0000-0000-01673807fd53', conn = conn)

1.25 ms ± 234 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
curs = conn.cursor()

# Check if sql vaccum helps

In [12]:
conn.execute("VACUUM")

In [13]:
%time load_by_run_spec(captured_run_id=1234, conn = conn)

Wall time: 17.3 s


results #1234@C:\Users\a-halakh\qt5-experiments-2019-08-29.db
-------------------------------------------------------------
scanner_b - array
scanner_pl - array
s_x - array
s_y - array

In [15]:
%time load_by_run_spec(captured_run_id=1234)

Wall time: 7.19 s


results #1234@C:\Users\a-halakh\qt5-experiments-2019-08-29.db
-------------------------------------------------------------
scanner_b - array
scanner_pl - array
s_x - array
s_y - array

In [16]:
%time curs.execute("VACUUM")

Wall time: 9min 45s


In [18]:
%time load_by_run_spec(captured_run_id=1234)

Wall time: 7.48 s


results #1234@C:\Users\a-halakh\qt5-experiments-2019-08-29.db
-------------------------------------------------------------
scanner_b - array
scanner_pl - array
s_x - array
s_y - array

In [19]:
%timeit load_by_id(1234)

250 ms ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit load_by_guid('aaaaaaaa-0000-0000-0000-01673807fd53')

237 ms ± 5.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Doing "Vaccum" do increase the efficiency of output. Still it can not reduce less that few hundred ms because the first the new connection takes that much time. 

# Profiling load_by_run_spec

In [21]:
import snakeviz

In [22]:
%load_ext snakeviz

In [23]:
%snakeviz load_by_run_spec(captured_run_id=1234)

 
*** Profile stats marshalled to file 'C:\\Users\\a-halakh\\AppData\\Local\\Temp\\tmpijau8v7j'. 
Embedding SnakeViz in this document...


In [24]:
rows = curs.fetchall()

In [25]:
len(rows)

0

# profiling

In [ ]:
from typing import Dict, List, Optional, Any, Sequence, Union, Tuple, \
    Callable, cast, Mapping

from qcodes.dataset.sqlite.queries import get_matching_exp_ids

In [ ]:
captured_run_id= 1234
captured_counter= None,
experiment_name = None,
sample_name = None

In [ ]:
exp_ids = None

conds = []
inputs = []


conds.append("captured_run_id is ?")
inputs.append(captured_run_id)

if len(conds) >= 1:
    where_clause = " WHERE " + " AND ".join(conds)
else:
    where_clause = ""

query = "SELECT guid from runs" + where_clause + " ORDER BY run_id"

cursor = conn.cursor()
if len(inputs) > 0:
    cursor.execute(query, inputs)
else:
    cursor.execute(query)

rows = cursor.fetchall()

In [ ]:
for r in rows:
    print(r['guid'])

In [ ]:
from qcodes.dataset.sqlite.connection import atomic_transaction, atomic, transaction
from tqdm import tqdm
import sys

In [ ]:

def perform_db_upgrade_8_to_9(conn) -> None:
    """
    Perform the upgrade from version 1 to version 2

    Add two indeces on the runs table, one for exp_id and one for GUID
    """

    sql = "SELECT name FROM sqlite_master WHERE type='table' AND name='runs'"
    cur = atomic_transaction(conn, sql)
    n_run_tables = len(cur.fetchall())

    pbar = tqdm(range(1), file=sys.stdout)
    pbar.set_description("Upgrading database; v1 -> v2")

    if n_run_tables == 1:
        _IX_runs_capture_id = """
                          CREATE INDEX
                          IF NOT EXISTS IX_runs_capture_id
                          ON runs (captured_run_id DESC)
                          """
        with atomic(conn) as conn:
            # iterate through the pbar for the sake of the side effect; it
            # prints that the database is being upgraded
            for _ in pbar:
                transaction(conn, _IX_runs_capture_id)
    else:
        raise RuntimeError(f"found {n_run_tables} runs tables expected 1")

In [ ]:
perform_db_upgrade_8_to_9(conn) 